# Récolte des débats à l'Assemblée Nationale  

Ce notebook présente les modalités pour automatiser la collecte des CRI (pour compte-rendus intégraux des débats en séance publique à l'Assemblée Nationale). Pour plus d'informations sur le choix et les différentes options relatives aux compte-rendus des débats en séance publiques à l'Assemblée Nationale française entre 2017 et 2024 (cf .../docs/Données_Débats_AN.md)

## Étape 1 : collecte des données [X]

### Pour l'option 1 : télécharger les xml

Lien d’accès :
- https://data.assemblee-nationale.fr/archives-anterieures/archives-15e/debats-en-seance-publique
- https://data.assemblee-nationale.fr/archives-16e/debats

### Pour l'option 2 : script de scrapping

Intérêt d'automatiser la récolte de tous les fichiers disponibles sur cette page 
- https://echanges.dila.gouv.fr/OPENDATA/Debats/AN/

In [3]:
import os
import requests
from bs4 import BeautifulSoup # nécessaire de l'installer avec pip la première fois.
from urllib.parse import urljoin

# 1/ Je crée un objet correspondant à l'url à partir duquel je souhaite scrapper les fichiers taz (à réaliser en changeant l'année pour chacune des années que je souhaite récupérer)
BASE_URL = "https://echanges.dila.gouv.fr/OPENDATA/Debats/AN/2018/"

# 2/ Je crée un objet pour indiquer le dossier (et son emplacement) où je souhaite déposer les fichiers récupérés 
destination = "/Users/matthiaslevalet/Desktop/Projet de recherche/CSS_République/Data/Raw/DILA/2018"

# 3/ J'écris une fonction pour aller récupérer tous les liens vers les fichiers taz depuis la page html/le lien url donné
def get_taz_links(base_url): #je définis par un nom qui correspond (aller récuper les liens taz avec entre parenthèse l'objet crée précédemment)
    response = requests.get(base_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser") #Beautifulsoup c'est le package que j'ai installé précédement (besoin de le faire qu'une seule fois donc présent ici dans un autre bloc de code) et qui permet le mieux de scrapper à partir d'url (cf le cours de Emilien et Léo sur Python)

    taz_links = []
    for link in soup.find_all("a"):
        href = link.get("href")
        if href and href.endswith(".taz"):
            full_url = urljoin(base_url, href)
            taz_links.append(full_url)

    return taz_links
# 4/ Je crée une fonction pour télécharger les fichiers depuis l'URL et les mettre dans le dossier défini à l'étape 2
def download_file(url, destination):
    local_filename = os.path.join(destination, url.split("/")[-1])

    if os.path.exists(local_filename):
        print(f"Déjà téléchargé : {local_filename}")
        return

    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

def main():
    os.makedirs(destination, exist_ok=True)
    print(f"Récupération des liens depuis {BASE_URL}...")
    taz_links = get_taz_links(BASE_URL)
    print(f"{len(taz_links)} fichiers trouvés.")

    for link in taz_links:
        download_file(link, destination)

if __name__ == "__main__":
    main()

# To do : créer une boucle pour le faire sur tous les fichiers de 2011 à 2024

Récupération des liens depuis https://echanges.dila.gouv.fr/OPENDATA/Debats/AN/2018/...
162 fichiers trouvés.


## Étape 2 : Décompresser les fichiers taz et répartir entre AAA et CRI (uniquement option 2)

2 problèmes rencontrés : 1 celui de la gestion et traitement de fichiers TAZ, et 2 même quand des XML obtenus, à l'exception de 2011-2013, je n'ai pas réussi à récupérer des fichiers de sorties propres. Il manque systèmatiquement les "retours à la ligne".


Comme écrit précédemment, sur la base de données du gouvernement géré par DILA, on trouve les débats sous formes de fichiers taz (ce sont des fichiers étrange, à la fois décompressé et difficilement ouvrable) comprenant chacun un AAA et un CRI. 
Ayant galéré à écrire un script qui fasse tout fonctionner, j'ai décomposé : 

1. Je transforme mes fichiers taz en tar pour avoir un fichier plus facilement exploitable
2. Je décompresse mes fichiers tar
3. Je déplace les fichiers AAA et CRI dans 2 dossiers séparés (où cette fois tous les CRI sont ensemble pour être mis dans un seul CSV)


### Partie 1 de l'étape 3 (transformer fichiers taz en tar)

Le premier script ne fonctionne que pour les fichiers de 2016 à 2024 puisque les fichiers de 2011 à 2015 sont compressés avec l’ancien format UNIX appelé compress qui utilise l’extension .Z. 

Plus précisément : 
| Année     | Contenu probable du `.taz`                   | Format interne    |
| --------- | -------------------------------------------- | ----------------- |
| 2011-2015 | `.Z` compressant un seul `.xml`              | **compress (.Z)** |
| 2016-2018 | `.Z` contenant un fichier `.tar` avec `.xml` | **tar.Z**         |
| 2019–2021 | `.gz` contenant un `.tar` → `.xml`           | **tar.gz**        |
| 2022–2024 | `.taz` ≈ `.xml` brut                         | **non compressé** |

==> Essayer de régler ces problèmes avec Léo (multiples tentatives sur le fichier test mais tous des échecs)

In [62]:
# Script pour les fichiers 2016-2024
# Ici ce que je fais, c'est que je transforme tous mes fichiers taz en fichiers tar (peut-il qu'il existe un moyen plus rapide de les traiter dont j'ai pas connaissance pour l'instant, à creuser)
import os # module pour gérer mon ordi
import tarfile # module pour gérer les fichiers tar

def extract_all_taz_as_tar(folder_path):

    if not os.path.isdir(folder_path):
        raise ValueError(f"Dossier introuvable : {folder_path}")

    for filename in os.listdir(folder_path):  
        if not filename.endswith(".taz"):
            continue

        file_path = os.path.join(folder_path, filename)
        print(f"Extraction de : {filename}")

        try:
            with tarfile.open(file_path, "r") as tar:
                tar.extractall(path=folder_path)
            print(f"Fichier extrait avec succès : {filename}")
        except Exception as e:
            print(f"Erreur lors de l'extraction de {filename} : {e}")

if __name__ == "__main__":
    dossier = "/Users/matthiaslevalet/Desktop/Try"
    extract_all_taz_as_tar(dossier)

# To do : créer une boucle pour répéter cette opération sur tous les sous-dossiers de 2016 à X (puisque réparti par année). 

Extraction de : AN_2022001.taz
Fichier extrait avec succès : AN_2022001.taz


/var/folders/gc/0c_w194n61ggswq_j8v40mj80000gn/T/ipykernel_1731/3817558419.py:20: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=folder_path)


In [ ]:
# Créer un autre script pour gérer le soucis des fichiers de 2011 à 2015 qui sont compressés différemment.  FONCTIONNE DE 2011 À 2013 mais problème pour les deux années suivantes où il n'y a pas de lignes terminator
# Il faut vérifier que ce soit le bon script qui a fonctionné, vérifier avec archives de chat
import subprocess
import shutil
import re
from pathlib import Path
import xml.etree.ElementTree as ET

def extract_xml_documents(content):
    """Extrait les blocs XML complets depuis du contenu concaténé."""
    pattern = re.compile(
        rb'(<\?xml[^>]*\?>.*?</[^>]+>)',
        re.DOTALL
    )
    return pattern.findall(content)

def extract_and_validate_documents(filepath, output_dir, base_name):
    try:
        with open(filepath, 'rb') as f:
            raw_data = f.read()

        # Nettoyage des caractères nuls
        cleaned_data = raw_data.replace(b'\x00', b'').strip()

        # Extraction des documents XML
        xml_blocks = extract_xml_documents(cleaned_data)

        if len(xml_blocks) < 2:
            print(f"[ERREUR] Moins de 2 documents XML trouvés dans {filepath.name}")
            return False

        xml1, xml2 = xml_blocks[:2]

        # Validation syntaxique
        for i, xml_bytes in enumerate([xml1, xml2], start=1):
            try:
                ET.fromstring(xml_bytes.decode('utf-8'))
            except ET.ParseError as e:
                print(f"[ERREUR XML] Document {i} mal formé : {e}")
                return False

        # Sauvegarde
        aaa_path = output_dir / f"AAA_{base_name}.xml"
        cri_path = output_dir / f"CRI_{base_name}.xml"

        with open(aaa_path, 'wb') as f1:
            f1.write(xml1)
        with open(cri_path, 'wb') as f2:
            f2.write(xml2)

        print(f"[OK] Extraction réussie : {aaa_path.name}, {cri_path.name}")
        return True

    except Exception as e:
        print(f"[ERREUR] Exception lors de l’extraction de {filepath.name} : {e}")
        return False

def decompress_taz_and_extract(source_dir, output_dir):
    source_dir = Path(source_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    for taz_file in source_dir.rglob("*.taz"):
        print(f"\n[INFO] Traitement : {taz_file.name}")

        temp_z_file = taz_file.with_suffix(".Z")
        try:
            shutil.copy2(taz_file, temp_z_file)
        except Exception as e:
            print(f"[ERREUR] Échec de la copie : {e}")
            continue

        # Décompression avec uncompress
        try:
            subprocess.run(["uncompress", str(temp_z_file)], check=True)
            decompressed_file = temp_z_file.with_suffix("")
            print(f"[OK] Décompressé : {decompressed_file.name}")
        except subprocess.CalledProcessError:
            print(f"[ERREUR] uncompress a échoué pour {taz_file.name}")
            temp_z_file.unlink(missing_ok=True)
            continue

        # Extraction et validation
        base_name = taz_file.stem
        success = extract_and_validate_documents(decompressed_file, output_dir, base_name)

        if success:
            decompressed_file.unlink(missing_ok=True)  # Supprimer le fichier temporaire


if __name__ == "__main__":
    dossier_source ="/Users/matthiaslevalet/Desktop/Try"        # ← À modifier
    dossier_sortie = "/Users/matthiaslevalet/Desktop/Try"     # ← À modifier
    decompress_taz_and_split_clean_xml(dossier_source, dossier_sortie)


[INFO] Traitement : AN_2022001.taz
[ERREUR] uncompress a échoué pour AN_2022001.taz


uncompress: /Users/matthiaslevalet/Desktop/Try/AN_2022001.Z: Inappropriate file type or format


In [ ]:
# Gérer le problème de 2014 et 2015 
    # Problème = absence de "line terminators". 
# Pour les solutions essayées voir notebook 2014-2015

### Partie 2 de l'étape 3 je décompresse les fichiers tar (pour les fichiers de 2016 à 2024, pour 2011-2013 pas besoin car déjà décompressés)

In [64]:
# Partie 2 : je décompresse les fichiers tar (pour les fichiers de 2016 à 2024, pour 2011-2013 pas besoin car déjà décompressés)
import os
import tarfile
from pathlib import Path

def decompress_tar_files(source_dir, target_dir):
    source_dir = Path(source_dir)
    target_dir = Path(target_dir)
    target_dir.mkdir(parents=True, exist_ok=True)

    for tar_path in source_dir.glob("*.tar"):
        # Crée un dossier de destination spécifique pour chaque archive
        extract_subdir = target_dir / tar_path.stem
        if extract_subdir.exists():
            print(f"[INFO] Skipping {tar_path.name}, already extracted.")
            continue

        print(f"[INFO] Extracting {tar_path.name} to {extract_subdir}...")
        extract_subdir.mkdir(exist_ok=True)

        try:
            with tarfile.open(tar_path, "r") as tar:
                tar.extractall(path=extract_subdir)
            print(f"[SUCCESS] Extracted: {tar_path.name}")
        except Exception as e:
            print(f"[ERROR] Failed to extract {tar_path.name}: {e}")

if __name__ == "__main__":
    source_folder = "/Users/matthiaslevalet/Desktop/Try"  # à modifier
    target_folder = "/Users/matthiaslevalet/Desktop/Try"   # à modifier
    decompress_tar_files(source_folder, target_folder)


[INFO] Extracting AN_2022001.tar to /Users/matthiaslevalet/Desktop/Try/AN_2022001...
[SUCCESS] Extracted: AN_2022001.tar


/var/folders/gc/0c_w194n61ggswq_j8v40mj80000gn/T/ipykernel_1731/2581559503.py:23: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=extract_subdir)


### Partie 3 de l'étape 3 : je trie les xml entre compte rendus et annexes

In [53]:

import os
import shutil
from pathlib import Path

def move_xml_files_by_prefix(source_dir, aaa_dir, cri_dir):
    source_dir = Path(source_dir)
    aaa_dir = Path(aaa_dir)
    cri_dir = Path(cri_dir)

    # Recherche dans tous les sous-dossiers
    for xml_file in source_dir.rglob("*.xml"):
        filename = xml_file.name

        if filename.startswith("AAA"):
            dest = aaa_dir / filename
        elif filename.startswith("CRI"):
            dest = cri_dir / filename
        else:
            print(f"[IGNORÉ] {filename} ne commence ni par AAA ni par CRI.")
            continue

        try:
            shutil.move(str(xml_file), str(dest))
            print(f"[DÉPLACÉ] {filename} → {dest}")
        except Exception as e:
            print(f"[ERREUR] Impossible de déplacer {filename} : {e}")

if __name__ == "__main__":
    source_folder = "/Users/matthiaslevalet/Desktop/Projet de recherche/CSS_République/Data/Raw/DILA/2011" 
    aaa_target = "/Users/matthiaslevalet/Desktop/Projet de recherche/CSS_République/Data/Interim/DILA/AAA" 
    cri_target = "/Users/matthiaslevalet/Desktop/Projet de recherche/CSS_République/Data/Interim/DILA/CRI"  

    move_xml_files_by_prefix(source_folder, aaa_target, cri_target)


[DÉPLACÉ] CRI_AN_20111025_094.xml → /Users/matthiaslevalet/Desktop/Projet de recherche/CSS_République/Data/Interim/DILA/CRI/CRI_AN_20111025_094.xml
[DÉPLACÉ] CRI_AN_20111214_122.xml → /Users/matthiaslevalet/Desktop/Projet de recherche/CSS_République/Data/Interim/DILA/CRI/CRI_AN_20111214_122.xml
[DÉPLACÉ] CRI_AN_20111123_111.xml → /Users/matthiaslevalet/Desktop/Projet de recherche/CSS_République/Data/Interim/DILA/CRI/CRI_AN_20111123_111.xml
[DÉPLACÉ] CRI_AN_20111012_087.xml → /Users/matthiaslevalet/Desktop/Projet de recherche/CSS_République/Data/Interim/DILA/CRI/CRI_AN_20111012_087.xml
[DÉPLACÉ] AAA_AN_20111223_128.xml → /Users/matthiaslevalet/Desktop/Projet de recherche/CSS_République/Data/Interim/DILA/AAA/AAA_AN_20111223_128.xml
[DÉPLACÉ] AAA_AN_20111208_120.xml → /Users/matthiaslevalet/Desktop/Projet de recherche/CSS_République/Data/Interim/DILA/AAA/AAA_AN_20111208_120.xml
[DÉPLACÉ] CRI_AN_20111115_106.xml → /Users/matthiaslevalet/Desktop/Projet de recherche/CSS_République/Data/Inter